# BoardGameGeek Recommender System

In [1]:
# Import Libraries
import pandas as pd
import numpy as np
from scipy import sparse
from sklearn.metrics.pairwise import cosine_distances

In [2]:
# Import data
rank_df = pd.read_csv('../data/ranked_data.csv', index_col=0).iloc[:2025]
rank_df.index = rank_df.collection_rank
cat_df = pd.read_csv('../data/cat_data.csv', index_col = 0)

# The recommender results are located within the recommender.csv so user_df is not needed

# # Due to memory allocation issues, all dtypes should be changed into 32 bit
# user_df = pd.read_csv('../data/user_ratings.csv').drop('Unnamed: 0', axis = 1)
# user_df['id'] = user_df['id'].astype(np.int32)
# user_df['username'] = user_df['username'].astype(np.int32)
# user_df['rating'] = user_df['rating'].astype(np.float32)

In [3]:
# Function to create a recommender model
def create_recommender():
    # Create pivot table
    pivot = user_df.pivot_table(index='game', columns='username', values='rating')

    # Create sparse matrix
    pivot_sparse = sparse.csr_matrix(pivot.fillna(0))

    # Create cosine distances based on pivot_sparse
    distances = cosine_distances(pivot_sparse)

    # Get distance values for recommender
    recommender = 1.0 - distances

    # Create recommender dataframe
    recommender_df = pd.DataFrame(recommender, index=pivot.index, columns=pivot.index)
    recommender_df.columns.name = None

    # Export model
    recommender_df.to_csv('../data/recommender.csv')

In [4]:
# Get recommender dataframe
recommender_df = pd.read_csv('../data/recommender.csv', index_col=0)

In [5]:
# Example of getting list of games from rank_df
search = input()
rank_df.loc[rank_df['name'].str.contains(search), 'name']

Gloomhaven


collection_rank
1                         Gloomhaven
5       Gloomhaven: Jaws of the Lion
1741          Founders of Gloomhaven
Name: name, dtype: object

In [6]:
# Function to get list of categories from cat_df
def get_cats(name):
    categories = cat_df[cat_df['name'] == name]['categories']
    new_list = []
    for cats in categories:
        test = cats.replace("'", '')
        test = test[1:-1].split(', ')
        for cat in test:
            if cat != 'N/A':
                new_list.append(cat)
        return new_list

# Function to take categories and input them into a string
def cats(name):
    categories = get_cats(name)
#     print(len(categories))
    sentence = ', '.join(categories)
    
    if len(categories) == 0:
        return (f'Sorry! {name} does not have any categories listed on BoardGameGeek.')
    else:
        return (f'The categories of {name} are:\n{sentence}')

In [7]:
cats('Gloomhaven')

'The categories of Gloomhaven are:\nAdventure, Exploration, Fantasy, Fighting, Miniatures'

In [8]:
# Recommender uses input to produce recommendations
def BGG_recommender(specific = False):
    # The recommender will use user input in order to search through rank_df
    print('Hello, please input a search term.\n')
    search = input()
    print('')

    # Accounts for wrong inputs and games that aren't in the dataframe
    if len(rank_df.loc[rank_df['name'].str.contains(search), 'name']) == 0:
            return('Sorry, there are no games containing that input!')
    
    # Shows top 10 game recommendations. If more are desired, change the sort_values index
    elif specific == False:
        for game_name in rank_df.loc[rank_df['name'].str.contains(search), 'name']:
            print(f'These are the top 10 recommendations for {game_name}.\n')
            print(recommender_df[game_name].sort_values(ascending=False)[1:11], '\n')
            
            # Displays categorical data of the search results, not the recommendations
            print(cats(game_name), '\n')
            print('----------------', '\n')
    
    # If you only want one game, specific should be True.
    else:
        print(f'These are the top 10 recommendations for {search}.\n')
        print(recommender_df[search].sort_values(ascending=False)[1:11], '\n')
            
        # Displays categorical data of the search results, not the recommendations
        print(cats(search), '\n')
        print('----------------', '\n')

In [ ]:
BGG_recommender()

Hello, please input a search term.

